# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Описание данных
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* purpose — цель получения кредита

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd

In [2]:
# посмотрим на данные
data = pd.read_csv('/datasets/data.csv')
data.head(20)
data.tail(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21506,1,-1556.249906,33,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145541.993320,свадьба
21507,1,-79.832064,32,среднее,1,гражданский брак,1,F,госслужащий,0,98180.279152,сделка с подержанным автомобилем
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21510,2,NaN,28,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,приобретение автомобиля
21511,0,-612.569129,29,высшее,0,гражданский брак,1,F,сотрудник,1,140068.472941,покупка жилья для сдачи
21512,0,-165.377752,26,высшее,0,Не женат / не замужем,4,M,компаньон,0,147301.457769,получение дополнительного образования
21513,0,-1166.216789,35,среднее,1,женат / замужем,0,F,сотрудник,0,250986.142309,покупка жилья
21514,0,-280.469996,27,неоконченное высшее,2,Не женат / не замужем,4,M,компаньон,0,355988.407188,строительство недвижимости


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Даю характеристику качеству данных
    
* 'children' - количественная переменная   
* 'days_emplpyed' - количественная переменная
* 'dob_years' - количественная переменная
* 'education' - категориальная переменная
* 'education_id' - категориальная переменная
* 'family_status' - категориальная переменная
* 'family_status_id' - категориальная переменная
* 'gender' - категориальная переменная
* 'income_type' - категориальная переменная
* 'debt' - логическая переменная
* 'total_income' - количественная переменная
* 'purpose' - категориальная переменная


## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
#data.nunique()
#print(data[data.isnull()].count())

In [5]:
#количество детей не может быть отрицательным и 20 детей большая редкость
#меняю на положительное, а строки с 20 детьми удаляю
#20 детей возможно получились слуайным умножением 5 и 4 столбцов
data['children'] = abs(data['children'])                      
data = data.drop(data[(data['children'] > 5)].index)
unique_children = data['children'].unique()
print(unique_children)

[1 0 3 2 4 5]


In [6]:
# количество дней не может быть отрицательным
# используем метод abs()
data['days_employed'] = abs(data['days_employed'])

In [7]:
#value_children = data['children'].value_counts()
#print(value_children)

In [8]:
#брать кредит можно только с 18 лет, удаляем строки с значениями меньше 18
#используем метод drop()
#значение 0 могло появиться в результате замены nan и none, то есть возраст этого человека был просто не определен 
data = data.drop(data[(data['dob_years'] < 18)].index)
unique_dob_years = data['dob_years'].unique()
print(unique_dob_years)

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51 59 29 60 55 58 71
 22 73 66 69 19 72 70 74 75]


In [9]:
#приводим значения столбца 'education' к нижнему регистру
#используем метод str.lower()
data['education'] = data['education'].str.lower()
unique_education = data['education'].unique()
print(unique_education)

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


In [10]:
#unique_education_id = data['education_id'].unique()
#print(unique_education_id)

In [11]:
#смотрим семейные положения людей
unique_family_status = data['family_status'].unique()
print(unique_family_status)
data.head(100)

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,541.832241,51,высшее,0,женат / замужем,0,F,сотрудник,0,94187.878082,автомобиль
96,0,NaN,44,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,покупка жилой недвижимости
97,0,NaN,47,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,профильное образование
98,0,364906.205736,54,высшее,0,женат / замужем,0,F,пенсионер,0,199707.298524,покупка жилья для сдачи


In [12]:
#unique_family_status_id = data['family_status_id'].unique()
#print(unique_family_status_id)

In [13]:
#удаляем строки с гендером XNA
#для поиска значение в столбце используем loc
data.drop(data.loc[data['gender']== 'XNA'].index, inplace=True)
unique_gender = data['gender'].unique()
print(unique_gender)

['F' 'M']


In [14]:
#смотрим какие социальный статусы существуют
unique_income_type = data['income_type'].unique()
print(unique_income_type)

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


**Вывод**

###### Удалили некорректные значения в стоблцах
###### Привели всё к нижнему регистру
###### Рассмотрели уникальные знаения в каждом столбце

### Замена типа данных

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21348 entries, 0 to 21524
Data columns (total 12 columns):
children            21348 non-null int64
days_employed       19193 non-null float64
dob_years           21348 non-null int64
education           21348 non-null object
education_id        21348 non-null int64
family_status       21348 non-null object
family_status_id    21348 non-null int64
gender              21348 non-null object
income_type         21348 non-null object
debt                21348 non-null int64
total_income        19193 non-null float64
purpose             21348 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [18]:
## Можем сразу обратить внимание на столбцы 'days_employed', 'total_income'
## Значения этих столбцов должны быть типа int, а не float, потому что дни и заработная плата измеряются в количественных единицах

In [19]:
# V3
# строим сводную таблицу и выделяем пропущенные значения в столбце 'days_employed'
table = pd.pivot_table(data, columns =['total_income', 'days_employed'])
bool_series = pd.isnull(data['days_employed'])
data[bool_series]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,высшее,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [21]:
#Для заполнения пропусков 'total_income' используем медиану для доходов людей схожих по типу занятости ('income_type')
data['total_income'].fillna(value = data.groupby(['income_type'])['total_income'].transform('median'), inplace = True)
#Для заполнения пропусков 'days_employed' используем медиану для общего трудового стажа в днях схожих по возрасту ('dob_years')
data['days_employed'].fillna(value = data.groupby(['dob_years'])['days_employed'].transform('median'), inplace = True)

In [22]:
data.isnull().sum(axis=0)

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [23]:
#Избавляемся от значений Nan, чтобы перевести столбцы в тип данных int
data = data.dropna()
data = data.reset_index(drop=True)
data['days_employed']=data['days_employed'].astype('int')
data['total_income']=data['total_income'].astype('int')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

#### Изменив тип данных, таблица стала более понятной

### Обработка дубликатов

In [24]:
print(data.duplicated().sum())

71


**Вывод**

### Дубликатов не найдено, потому что мы от них избавились на этапе обработки пропусков

### Лемматизация

In [25]:
# импортируем нужные библиотеки для лемматизации
# создаём функцию лемматизации
from pymystem3 import Mystem
from collections import Counter
def lemat (table):
    m = Mystem()
    data_purpose = table
    row = ' '.join(data_purpose)
    lemmas = m.lemmatize(row)
    print(Counter(lemmas)) 

lemat(data['purpose'])




## импортируем нужные библиотеки для лемматизации
## значения столбца переведём в список и разделим значения пробелом
#from pymystem3 import Mystem
#from collections import Counter
#m = Mystem()
#dat[] = 
#data_purpose = data['purpose']
#row = ' '.join(data_purpose)
#lemmas = m.lemmatize(row)
#print(Counter(lemmas)) 

Counter({' ': 54766, 'недвижимость': 6322, 'покупка': 5866, 'жилье': 4434, 'автомобиль': 4278, 'образование': 3989, 'с': 2897, 'операция': 2589, 'свадьба': 2325, 'свой': 2224, 'на': 2211, 'строительство': 1869, 'высокий': 1361, 'получение': 1308, 'коммерческий': 1303, 'для': 1289, 'жилой': 1224, 'сделка': 937, 'заниматься': 904, 'дополнительный': 901, 'подержать': 845, 'проведение': 769, 'сыграть': 765, 'сдача': 650, 'семья': 639, 'со': 629, 'собственный': 627, 'ремонт': 608, 'приобретение': 460, 'профильный': 433, 'подержанный': 111, '\n': 1})


**Вывод**

### Наиболее популярные цели кредитов:
#### 1. Недвижимость
#### 2. Жильё
#### 3. Автомобиль
#### 4. Образование
#### 5. Свадьба

### Категоризация данных

In [26]:
data_dict = data[['children','family_status']]
print(data_dict.sort_values('children', ascending = True))

       children          family_status
10673         0       гражданский брак
10048         0        женат / замужем
17437         0  Не женат / не замужем
10050         0        женат / замужем
10051         0  Не женат / не замужем
...         ...                    ...
15779         5        женат / замужем
4366          5        женат / замужем
20284         5        женат / замужем
7809          5        женат / замужем
16072         5       гражданский брак

[21348 rows x 2 columns]


In [27]:
# функция сортировки по возрасту
def age_group(age):

    if age < 18:
            return 'дети'
    if 18 < age <= 64:
            return 'взрослые'
    return 'пенсионеры'


In [28]:
#функия сортировки по целям
def purpose_group(purpose):
    if 'недвижимость' in purpose:
        return 'коммерция'
    if 'жилье' in purpose:
        return 'ипотека'
    if 'автомобиль' in purpose:
        return 'автокредит'
    if 'образование' in purpose:    # добавил ещё одну категорию
        return 'кредит на образование'
    if 'свадьба' in purpose:
        return 'кредит'

In [29]:
#print(data['income_type'].unique())
#функция сортировки по статусу
def work_status_group(status):
    if ('сотрудник' or 'госслужащий')  in status:
        return 'рабочий класс'
    if ('студент' or 'безработный') in status:
        return 'нерабочий класс'
    if ('пенсионер' or 'в декрете') in status:
        return 'нерабочий класс'
    if ('компаньон' or 'предприниматель') in status:
        return 'ни то и не другое'

In [30]:
#функия сортировки по ежемесячному доходу
def total_income_group(total_income):
    if total_income >= 98000:
        return 'выше среднего'
    if 88000 <= total_income < 98000:
        return 'средняя'
    if total_income < 88000:
        return 'ниже среднего'
    

In [31]:
#применяем написанные нами функции к столбцам
data['age_group'] = data['dob_years'].apply(age_group)
print(data['age_group'].value_counts())
print('\n')

data['purpose_group'] = data['purpose'].apply(purpose_group)
print(data['purpose_group'].value_counts())
print('\n')

data['work_status_group'] = data['income_type'].apply(work_status_group)
print(data['work_status_group'].value_counts())
print('\n')

data['total_income_group'] = data['total_income'].apply(total_income_group)
print(data['total_income_group'].value_counts())

взрослые      20450
пенсионеры      898
Name: age_group, dtype: int64


кредит на образование    2681
коммерция                2575
ипотека                  1280
автокредит                964
кредит                    791
Name: purpose_group, dtype: int64


рабочий класс        11022
ни то и не другое     5042
нерабочий класс       3828
Name: work_status_group, dtype: int64


выше среднего    17154
ниже среднего     3120
средняя           1074
Name: total_income_group, dtype: int64


**Вывод**

###### Категоризация по возрасту людей:
###### Разделили всех на детей, взрослых и пенсионеров
###### Категоризация по целям кредита:
###### разделили между видами кредитов: коммерческий, ипотека, автокредит, потребительский кредит 
###### Категоризация по социальному статусу:
###### Предпринимателей не отношу к рабочему классу по причине нестабильной экономической ситуации в разгар пандемии, данные могли устареть, много ИП-ей закрылись, многие из остальных получают льготы
###### Разделили на рабочий, нерабочий и предпринимателей
###### Категоризация по ежемесячному доходу:
###### Выбрал данные о средней заработной плате с сайта Ростата
###### Разделили на средний класс и на тех кто меньше и больше зарабатывает

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [32]:
# создаём сводную таблицу
# столбец 'percent' показывает с какой вероятностью люди не выплачивают кредит
children_task3 =  data.groupby('children').agg(count = ('debt','count'), debt_count = ('debt', 'sum'))
children_task3['percent'] = children_task3['debt_count'] / children_task3['count'] 
children_task3['percent'] = children_task3['percent'].map('{:.2%}'.format)
children_task3

,count,debt_count,percent
children,,,
0,14079,1058,7.51%
1,4849,442,9.12%
2,2042,194,9.50%
3,328,27,8.23%
4,41,4,9.76%
5,9,0,0.00%


**Вывод**

##### наличие детей увеличивает риск задолженности по кредиту. Недостаточно выборки для строки с 5 детьми. Результат того, что люди с 5 детьми не имеют долгов является неверным.

**Вывод**

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
# создаём сводную таблицу
family_status_pivot = data.pivot_table(index='family_status', values='debt', aggfunc=['mean'])
family_status_pivot 

,mean
,debt
family_status,
Не женат / не замужем,0.097561
в разводе,0.071006
вдовец / вдова,0.065195
гражданский брак,0.092445
женат / замужем,0.075226


###### 'вдовец / вдова'  и 'в разводе' имеют меньше долгов в сравнении с другими. Когда человек разводится, то все долги делятся пополам, возможно это упрощает выплаты. 'вдовец / вдова' имеет самый низкий показатель, могу предположить, что после смерти супруга, супруги продаётся его/её имущество, что покрывает часть выплат. После гибели супруга / супруги возможны выплаты страховых компаний. Больше всего рисков невозврата кредитов у 'Не женат / не замужем' и 'гражданский брак', возможно потому что одному оплачивать кредиты сложнее.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
# используем apply(), чтобы применить функцию к столбцу
# столбец 'percent' показывает с какой вероятностью люди не выплачивают кредит
data['total_incomemm'] = data['total_income'].apply(total_income_group)
total_income_task3 =  data.groupby('total_incomemm').agg(count = ('debt','count'), debt_count = ('debt', 'sum'))
total_income_task3['percent'] = total_income_task3['debt_count'] / total_income_task3['count'] 
total_income_task3['percent'] = total_income_task3['percent'].map('{:.2%}'.format)
total_income_task3

,count,debt_count,percent
total_incomemm,,,
выше среднего,17154,1385,8.07%
ниже среднего,3120,240,7.69%
средняя,1074,100,9.31%


**Вывод**

###### Средний класс чаще не выплачивает долги, чем люди, которые меньше или больше зарабатывают.

- Как разные цели кредита влияют на его возврат в срок?

In [35]:
# создаём сводную таблицу
purpose_task3 = data.pivot_table(index='purpose_group', values='debt', aggfunc=['mean'])
purpose_task3

,mean
,debt
purpose_group,
автокредит,0.090249
ипотека,0.072656
коммерция,0.076893
кредит,0.079646
кредит на образование,0.088027


**Вывод**

###### Люди, берущие кредит на жильё, свадьбу, недвижимость гораздо чаще возвращают долги нежели люди с кредитами на образование, автомобиль

## Шаг 4. Общий вывод

 ***Влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок?***
 * Количество детей не влияет на факт погашения кредита в срок, но отсутствие детей уменьшает риск невозврата кредитов.
 * Семейное положение влияет на факт погашения кредита в срок. 'вдовец / вдова' и 'в разводе' меньше всех имеют задолженности. 'женат/замужем' чуть менее надёжны чем 'вдовец / вдова' и 'в разводе'. Чаще **не** возвращают долги клиенты в гражданском браке или 'не женаты / не замужем'. 'не женаты / не замужем' показывает самый плохой результат.